# **Data Collection Notebook**

## Objectives

* Write here your notebook objective, for example, "Fetch data from Kaggle and save as raw data", or "engineer features for modelling"

## Inputs

* Write here which data or information you need to run the notebook 

## Outputs

* Write here which files, code or artefacts you generate by the end of the notebook 

## Additional Comments

* In case you have any additional comments that don't fit in the previous bullets, please state them here. 



---

Initial plan EDIT:

1:
Import packages
Install Kaggle %pip install kaggle==1.5.12

Get data from Kaggle
os.environ['KAGGLE_CONFIG_DIR'] = os.getcwd()
! chmod 600 kaggle.json
KaggleDatasetPath = "codeinstitute/malaria-cell-classification" EDIT
DestinationFolder = "inputs/malaria_dataset"   
! kaggle datasets download -d {KaggleDatasetPath} -p {DestinationFolder}
unzip in destination
remove zip file

2:
DATA CLEANING
Remove any non image files see walkthgouh1
Split train validation test sets according to conventions 
Conventionally,
* The training set is divided into a 0.70 ratio of data.
* The validation set is divided into a 0.10 ratio of data.
* The test set is divided into a 0.20 ratio of data.

In [ ]:
# import os
# import numpy as np
# import pandas as pd
# from PIL import Image
# import matplotlib.pyplot as plt
# import seaborn as sns
# from sklearn.model_selection import train_test_split
# from joblib import dump, load
# import zipfile
# from typing import Tuple, List, Dict
# import logging

# class LeafImageProcessor:
#     def __init__(self, kaggle_dataset: str, destination_folder: str):
#         """
#         Initialize the image processor with dataset paths
        
#         Args:
#             kaggle_dataset: Kaggle dataset path
#             destination_folder: Local destination for dataset
#         """
#         self.kaggle_dataset = kaggle_dataset
#         self.destination_folder = destination_folder
#         self.setup_logging()
        
#     def setup_logging(self):
#         """Configure logging for the pipeline"""
#         logging.basicConfig(
#             level=logging.INFO,
#             format='%(asctime)s - %(levelname)s - %(message)s',
#             handlers=[
#                 logging.FileHandler('image_processing.log'),
#                 logging.StreamHandler()
#             ]
#         )
        
#     def load_kaggle_data(self) -> None:
#         """Download and extract Kaggle dataset"""
#         try:
#             # Ensure Kaggle credentials are properly set
#             os.environ['KAGGLE_CONFIG_DIR'] = os.getcwd()
#             os.chmod('kaggle.json', 0o600)
            
#             # Download dataset
#             os.system(f'kaggle datasets download -d {self.kaggle_dataset} -p {self.destination_folder}')
            
#             # Extract dataset
#             zip_files = [f for f in os.listdir(self.destination_folder) if f.endswith('.zip')]
#             for zip_file in zip_files:
#                 zip_path = os.path.join(self.destination_folder, zip_file)
#                 with zipfile.ZipFile(zip_path, 'r') as zip_ref:
#                     zip_ref.extractall(self.destination_folder)
#                 os.remove(zip_path)
                
#             logging.info("Dataset successfully downloaded and extracted")
            
#         except Exception as e:
#             logging.error(f"Error in data loading: {str(e)}")
#             raise
            
#     def clean_data(self) -> Tuple[List[str], List[str]]:
#         """
#         Clean data directory and split into train/val/test sets
        
#         Returns:
#             Tuple of training and validation file paths
#         """
#         try:
#             # Get all image files
#             image_files = []
#             for root, _, files in os.walk(self.destination_folder):
#                 for file in files:
#                     if file.lower().endswith(('.png', '.jpg', '.jpeg')):
#                         image_files.append(os.path.join(root, file))
                        
#             # Remove non-image files
#             for root, _, files in os.walk(self.destination_folder):
#                 for file in files:
#                     if not file.lower().endswith(('.png', '.jpg', '.jpeg')):
#                         os.remove(os.path.join(root, file))
                        
#             # Split data
#             train_files, test_files = train_test_split(image_files, test_size=0.2, random_state=42)
#             train_files, val_files = train_test_split(train_files, test_size=0.125, random_state=42)  # 0.125 of 80% = 10% of total
            
#             logging.info(f"Data split complete: {len(train_files)} train, {len(val_files)} val, {len(test_files)} test")
#             return train_files, val_files
            
#         except Exception as e:
#             logging.error(f"Error in data cleaning: {str(e)}")
#             raise
            
#     def analyze_image_statistics(self, image_files: List[str]) -> Dict:
#         """
#         Compute image statistics
        
#         Args:
#             image_files: List of image file paths
            
#         Returns:
#             Dictionary of image statistics
#         """
#         try:
#             sizes = []
#             for img_path in image_files:
#                 with Image.open(img_path) as img:
#                     sizes.append(img.size)
                    
#             sizes = np.array(sizes)
#             stats = {
#                 'mean_width': np.mean(sizes[:, 0]),
#                 'mean_height': np.mean(sizes[:, 1]),
#                 'std_width': np.std(sizes[:, 0]),
#                 'std_height': np.std(sizes[:, 1])
#             }
            
#             logging.info(f"Image statistics computed: {stats}")
#             return stats
            
#         except Exception as e:
#             logging.error(f"Error in computing image statistics: {str(e)}")
#             raise
            
#     def create_visualizations(self, train_files: List[str], save_dir: str = 'outputs/visualizations'):
#         """
#         Create and save visualizations
        
#         Args:
#             train_files: List of training image files
#             save_dir: Directory to save visualizations
#         """
#         try:
#             os.makedirs(save_dir, exist_ok=True)
            
#             # Average image per class
#             healthy_imgs = [img for img in train_files if 'healthy' in img.lower()]
#             infected_imgs = [img for img in train_files if 'powdery_mildew' in img.lower()]
            
#             def compute_average_image(file_list):
#                 images = []
#                 target_size = (100, 100)
#                 for f in file_list[:100]:  # Limit to prevent memory issues
#                     with Image.open(f) as img:
#                         img_resized = img.resize(target_size)
#                         images.append(np.array(img_resized))
#                 return np.mean(images, axis=0)
            
#             # Create visualization plots
#             avg_healthy = compute_average_image(healthy_imgs)
#             avg_infected = compute_average_image(infected_imgs)
#             difference = avg_infected - avg_healthy
            
#             # Plot and save
#             fig, axes = plt.subplots(1, 3, figsize=(15, 5))
#             axes[0].imshow(avg_healthy)
#             axes[0].set_title('Average Healthy Leaf')
#             axes[1].imshow(avg_infected)
#             axes[1].set_title('Average Infected Leaf')
#             axes[2].imshow(difference)
#             axes[2].set_title('Difference')
#             plt.savefig(os.path.join(save_dir, 'average_images.png'))
            
#             logging.info("Visualizations created and saved")
            
#         except Exception as e:
#             logging.error(f"Error in creating visualizations: {str(e)}")
#             raise
            
#     def save_artifacts(self, stats: Dict, save_dir: str = 'outputs/artifacts'):
#         """
#         Save processing artifacts
        
#         Args:
#             stats: Dictionary of computed statistics
#             save_dir: Directory to save artifacts
#         """
#         try:
#             os.makedirs(save_dir, exist_ok=True)
            
#             # Save statistics
#             dump(stats, os.path.join(save_dir, 'image_statistics.pkl'))
            
#             # Save summary as text
#             with open(os.path.join(save_dir, 'processing_summary.txt'), 'w') as f:
#                 f.write(f"Image Processing Summary\n{'='*20}\n")
#                 for key, value in stats.items():
#                     f.write(f"{key}: {value}\n")
                    
#             logging.info(f"Artifacts saved to {save_dir}")
            
#         except Exception as e:
#             logging.error(f"Error in saving artifacts: {str(e)}")
#             raise

# # Example usage
# if __name__ == "__main__":
#     processor = LeafImageProcessor(
#         kaggle_dataset="codeinstitute/cherry-leaves",  # Update with correct dataset
#         destination_folder="inputs/cherry_leaves"
#     )
    
#     # Execute pipeline
#     processor.load_kaggle_data()
#     train_files, val_files = processor.clean_data()
#     stats = processor.analyze_image_statistics(train_files)
#     processor.create_visualizations(train_files)
#     processor.save_artifacts(stats)

Possible enhancements to initial workflow:


Data Loading:

Added error handling for Kaggle API
Included validation of downloaded files
Added logging for tracking download progress


Data Cleaning:

Automated removal of non-image files
Implemented proper train/val/test split with fixed random seed
Added validation of image integrity


Data Visualization:

Enhanced image statistics computation
Added memory-efficient processing for large datasets
Included proper error handling for corrupt images


Additional Features:

Logging system for tracking processing steps
Artifact management for saving results
Type hints for better code maintainability



Key improvements over your initial plan:

Error Handling:

Robust error checking at each step
Graceful handling of corrupt images


Memory Management:

Batch processing for large datasets
Efficient image loading and processing


Documentation:

Clear logging of each step
Comprehensive statistics saved as artifacts


Extensibility:

Easy to add new visualization types
Modular design for adding new processing steps

# Change working directory

* We are assuming you will store the notebooks in a subfolder, therefore when running the notebook in the editor, you will need to change the working directory

We need to change the working directory from its current folder to its parent folder
* We access the current directory with os.getcwd()

In [ ]:
import os
current_dir = os.getcwd()
current_dir

We want to make the parent of the current directory the new current directory
* os.path.dirname() gets the parent directory
* os.chir() defines the new current directory

In [ ]:
os.chdir(os.path.dirname(current_dir))
print("You set a new current directory")

Confirm the new current directory

In [ ]:
current_dir = os.getcwd()
current_dir

# Section 1

Section 1 content

---

# Section 2

Section 2 content

---

NOTE

* You may add as many sections as you want, as long as it supports your project workflow.
* All notebook's cells should be run top-down (you can't create a dynamic wherein a given point you need to go back to a previous cell to execute some task, like go back to a previous cell and refresh a variable content)

---

# Push files to Repo

* If you don't need to push files to Repo, you may replace this section with "Conclusions and Next Steps" and state your conclusions and next steps.

In [ ]:
import os
try:
    # create here your folder
    # os.makedirs(name='')
except Exception as e:
    print(e)
